Christian Hansen  
LSTM Practice  

reference: http://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import sys
from sklearn.feature_extraction.text import TfidfVectorizer

Using TensorFlow backend.


In [2]:
# load ascii text and covert to lowercase
filename = "./text/sid.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [3]:
# mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [4]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [5]:
n_chars = len(raw_text)
n_vocab = len(chars)
print( "Total Characters: ", n_chars)
print( "Total Vocab: ", n_vocab)

Total Characters:  236854
Total Vocab:  60


In [6]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print( "Total Patterns: ", n_patterns)

Total Patterns:  236754


In [7]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [8]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [9]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [19]:
model.fit(X, y,nb_epoch=3, batch_size=128, callbacks=callbacks_list)

Epoch 1/3


ResourceExhaustedError: OOM when allocating tensor with shape[100,128,1024]
	 [[Node: gradients/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3_grad/TensorArrayGatherV3 = TensorArrayGatherV3[_class=["loc:@TensorArray_1"], dtype=DT_FLOAT, element_shape=<unknown>, _device="/job:localhost/replica:0/task:0/gpu:0"](gradients/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3_grad/TensorArrayGrad/TensorArrayGradV3, TensorArrayUnstack/range, gradients/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3_grad/TensorArrayGrad/gradient_flow)]]

Caused by op 'gradients/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3_grad/TensorArrayGatherV3', defined at:
  File "/usr/lib64/python3.4/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib64/python3.4/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/lib/python3.4/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "

## GENERATING TEXT

In [10]:
!ls

LSTM_Story.ipynb		    weights-improvement-00-2.9020.hdf5
nohup.out			    weights-improvement-00-2.9090.hdf5
nohup.txt			    weights-improvement-01-2.0850.hdf5
text				    weights-improvement-01-2.1235.hdf5
untitled.txt			    weights-improvement-01-2.4926.hdf5
weights-improvement-00-2.1034.hdf5  weights-improvement-01-2.6176.hdf5
weights-improvement-00-2.1434.hdf5  weights-improvement-02-2.0674.hdf5
weights-improvement-00-2.1654.hdf5  weights-improvement-02-2.4403.hdf5
weights-improvement-00-2.5509.hdf5  weights-improvement-03-2.3927.hdf5
weights-improvement-00-2.5517.hdf5  weights-improvement-04-2.3509.hdf5
weights-improvement-00-2.6820.hdf5  weights-improvement-05-2.3120.hdf5
weights-improvement-00-2.6845.hdf5  weights-improvement-06-2.2766.hdf5
weights-improvement-00-2.7622.hdf5  weights-improvement-07-2.2451.hdf5
weights-improvement-00-2.7642.hdf5  weights-improvement-08-2.2163.hdf5
weights-improvement-00-2.8965.hdf5  weights-improvement-09-2.1898.hdf5
weights-improvement-00-2.9012.

In [11]:
# load the network weights
filename =  "weights-improvement-02-2.0674.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [12]:
#get values back from char
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [18]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print( "Seed:")
print( "\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(100):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose = 0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print( "\nDone.")

Seed:
" cks, online payments and credit card donations.
to donate, please visit: http://pglaf.org/donate


s "
oetes work oe aoe the pooject gutenberg ti enectronic works in aoy ptrenc oo aey porje tfe project g
Done.
